In [1]:
import time
import xarray as xr
import numpy as np

from xcube.core.rectify import rectify_dataset
from xcube.core.sentinel3 import open_sentinel3_product
from xcube.util.perf import measure_time

Define a fake-logger. The normal logger output does not appear in notebooks.

In [2]:
class Logger:
    info = print
logger = Logger()

## Rectify an OCLI L1b Product

---
Open a Sentinel-3 OLCI L1b product:

In [3]:
src_ds = open_sentinel3_product('C:\\Users\\Norman\\Downloads\\S3B_OL_1_EFR____20190728T103451_20190728T103751_20190729T141105_0179_028_108_1800_LN1_O_NT_002.SEN3')

NotADirectoryError: [Errno 20] Not a directory: '/home/alicja/Desktop/testdir/cyano_shift/test_cyano_2/L2_of_S3A_OL_1_EFR____20190820T093706_20190820T094006_20190821T160254_0179_048_193_1800_MAR_O_NT_002.SEN3.nc'

In [ ]:
src_ds

In [ ]:
src_ds.Oa13_radiance.plot.imshow(figsize=(16,16), vmax=40, cmap='Greys_r')

Preload x,y coordinates so we are faster:

In [ ]:
src_ds.longitude.load()
src_ds.latitude.load()

In [ ]:
src_ds.longitude.diff('rows').plot.imshow(figsize=(16,16), vmin=-0.005, vmax=0.005)

In [ ]:
src_ds.longitude.diff('columns').plot.imshow(figsize=(16,16), vmin=-0.01, vmax=0.01)

In [ ]:
src_ds.latitude.diff('rows').plot.imshow(figsize=(16,16), vmin=-0.003, vmax=-0.002)

In [ ]:
src_ds.latitude.diff('columns').plot.imshow(figsize=(16,16), vmin=-0.002, vmax=0.0005)

In [ ]:
col_diff = np.sqrt(np.square(src_ds.longitude.diff('columns')) + np.square(src_ds.latitude.diff('columns'))) 
col_diff

In [ ]:
row_diff = np.sqrt(np.square(src_ds.longitude.diff('rows')) + np.square(src_ds.latitude.diff('rows'))) 
row_diff

In [ ]:
col_diff.plot.imshow(figsize=(16,16), vmin=0.0, vmax=0.01)

In [ ]:
row_diff.plot.imshow(figsize=(16,16), vmin=0.003, vmax=0.005)

---
Rectify the dataset:

In [ ]:
with measure_time(tag='reproject_dataset', logger=logger):
    dst_ds = rectify_dataset(src_ds, var_names='Oa13_radiance')

In [ ]:
dst_ds

In [ ]:
Oa13_radiance = dst_ds.Oa13_radiance

In [ ]:
Oa13_radiance.plot.imshow(figsize=(16,16), vmax=40, cmap='Greys_r')

## Creating subsets from source `longitude` and `latitude` using a bounding box `bbox`

Bounding box `bbox` in decimal degree:

In [ ]:
bbox = (5, 62, 7, 64)

Compute `src_bbox` by selecting pixels in `bbox`:

In [ ]:
src_bbox = np.logical_and(np.logical_and(src_ds.longitude >= bbox[0], src_ds.longitude <= bbox[2]), 
                          np.logical_and(src_ds.latitude >= bbox[1], src_ds.latitude <= bbox[3]))
src_bbox.plot.imshow()

Select pixel coordinates from coordinate variables `x`, `y` using selected pixels in `src_bbox`:

In [ ]:
x = src_ds.columns.where(src_bbox)
y = src_ds.rows.where(src_bbox)

In [ ]:
x.plot.imshow()

In [ ]:
y.plot.imshow()

Get new bouning box `(x1, y1, x2, y2)` in pixel coordinates from min/max of `x` and `y`:

In [ ]:
x1 = int(x.min())
x2 = int(x.max())
y1 = int(y.min())
y2 = int(y.max())

In [ ]:
x1, y1, x2, y2

Use bounding box `(x1, y1, x2, y2)` to select subset within source image:

In [ ]:
rad = src_ds.Oa13_radiance.isel(columns=slice(x1, x2 + 1), rows=slice(y1, y2 + 1))
rad

In [ ]:
rad.plot.imshow(figsize=(10,10), vmax=40, cmap='Greys_r')

Verify: use bounding box `bbox` to select subset within target image:

In [ ]:
rad = dst_ds.Oa13_radiance.sel(longitude=slice(bbox[0], bbox[2]), latitude=slice(bbox[1], bbox[3]))
rad

In [ ]:
rad.plot.imshow(figsize=(10,10), vmax=40, cmap='Greys_r')